# Install Dependencies

In [2]:
%pip uninstall -y \
  langchain langchain-core langchain-community langchain-text-splitters \
  langchain-groq langsmith numpy pandas

%pip install --no-cache-dir \
  numpy==1.26.4 \
  pandas==2.2.1 \
  scikit-learn==1.4.2 \
  faiss-cpu==1.8.0 \
  sentence-transformers==2.6.1 \
  transformers==4.38.2 \
  pymupdf==1.23.26 \
  rank_bm25==0.2.2 \
  langchain==0.1.20 \
  langchain-core==0.1.53 \
  langchain-community==0.0.38 \
  langchain-text-splitters==0.0.1 \
  groq


Found existing installation: langchain 1.2.8
Uninstalling langchain-1.2.8:
  Successfully uninstalled langchain-1.2.8
Found existing installation: langchain-core 1.2.9
Uninstalling langchain-core-1.2.9:
  Successfully uninstalled langchain-core-1.2.9
Found existing installation: langsmith 0.6.9
Uninstalling langsmith-0.6.9:
  Successfully uninstalled langsmith-0.6.9
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 257.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 187.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

# Import Libraries

In [5]:
import fitz  # PyMuPDF
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import ndcg_score

from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter
)

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

from langchain.retrievers import EnsembleRetriever, ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
#from langchain_ollama import OllamaLLM, ChatOllama #(use accordingly local ollama or cloud based llama deployment)
from groq import Groq
from sklearn.metrics.pairwise import cosine_similarity

import json
import re

import warnings
warnings.filterwarnings("ignore")

import os
import logging

os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

logging.getLogger().setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("huggingface_hub").setLevel(logging.ERROR)
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)
logging.getLogger("langchain").setLevel(logging.ERROR)
logging.getLogger("langchain_text_splitters").setLevel(logging.ERROR)


# PDF Text Extraction

In [6]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        blocks = page.get_text("blocks")
        for block in blocks:
            if len(block) > 4 and block[4].strip():
                text += block[4] + "\n\n"
    doc.close()
    return text.strip()

pdf_path = "sample-service-manual 1.pdf"
full_text = extract_text_from_pdf(pdf_path)
print(f"Extracted text length: {len(full_text)}")
print(f"First 500 chars: {full_text[:500]}")  # Checking whether file loaded properly

Extracted text length: 883449
First 500 chars: Suspension System 


Inspection and Verification 


1.
Road test. 


z Verify the customer concern by carrying out a road test on a smooth road. If any vibrations are 


apparent, refer to Section 100-04 . 


2.
Inspect tires. 


z Check the tire pressure with all normal loads in the vehicle and the tires cold. Refer to the 


Vehicle Certification (VC) label. 


z Verify that all tires are sized to specification. Refer to the VC label. 


z Inspect the tires for incorrect wear and damage. Insta


# Embedding Model
Using Sentence-Transformers for embeddings (all-MiniLM-L6-v2).

In [7]:
embedding_model = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

# Chunking Methods
## We compare three chunking methods:

1) Fixed-size chunks (simple, fast).
2) Recursive character splitting (handles logical breaks).
3) Semantic chunking (using MiniLM to split).

In [9]:
def fixed_chunking(text, size=500, overlap=50):
    return CharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=overlap
    ).split_text(text)


def recursive_chunking(text, size=500, overlap=50):
    return RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=overlap
    ).split_text(text)


def semantic_chunking(text, sim_threshold=0.75, min_len=120):
    sentences = [s.strip() for s in text.split(".") if s.strip()]
    if not sentences:
        return []

    embeddings = embedding_model.embed_documents(sentences)

    chunks = []
    current_chunk = [sentences[0]]

    for i in range(1, len(sentences)):
        sim = cosine_similarity(
            [embeddings[i-1]],
            [embeddings[i]]
        )[0][0]

        if sim >= sim_threshold:
            current_chunk.append(sentences[i])
        else:
            chunk_text = ". ".join(current_chunk)
            if len(chunk_text) >= min_len:
                chunks.append(chunk_text)
            current_chunk = [sentences[i]]

    # last chunk
    chunk_text = ". ".join(current_chunk)
    if len(chunk_text) >= min_len:
        chunks.append(chunk_text)

    return chunks


chunking_methods = {
    "fixed": fixed_chunking,
    "recursive": recursive_chunking,
    "semantic": semantic_chunking
}


# Define Retrievers
Compare:

1) Sparse: BM25.
Dense:
2) FAISS vector search.
3) Hybrid: Ensemble of BM25 + Dense.
4) With Re-ranker: Cross-encoder for re-ranking top-k results.

In [11]:
def create_bm25(chunks):
    return BM25Retriever.from_texts(chunks, k=5)
def create_dense(chunks):
    vectorstore = FAISS.from_texts(chunks, embedding_model)
    return vectorstore.as_retriever(search_kwargs={"k": 5})
def create_hybrid(bm25, dense):
    return EnsembleRetriever(
        retrievers=[bm25, dense],
        weights=[0.5, 0.5]
    )
def create_reranked(hybrid_retriever, top_n=5):
    cross_encoder = HuggingFaceCrossEncoder(
        model_name="cross-encoder/ms-marco-MiniLM-L-6-v2"
    )

    compressor = CrossEncoderReranker(
        model=cross_encoder,
        top_n=top_n
    )

    return ContextualCompressionRetriever(
        base_retriever=hybrid_retriever,
        base_compressor=compressor
    )


# Queries and Ground Truth (Keyword-Based)

**15 technical service manual queries** with keyword-based relevance criteria. Each query maps to specific component keywords and torque values. The `get_relevant_indices()` function identifies ground-truth chunks by requiring all keywords to be present.

In [12]:
# ============================================================
# FEW HARD-CODED QUERIES + KEYWORDS (from Service Manual) FOR TESTING
# Used to evaluate: chunking × retriever
# ============================================================

queries = [
    "Torque for lower arm forward and rearward nuts",
    "Torque for lower ball joint nut",
    "Torque for upper ball joint nut",
    "Torque for shock absorber lower nuts",
    "Torque for shock absorber upper mount nuts",
    "Torque for shock rod nut",
    "Torque for stabilizer bar bracket nuts",
    "Torque for stabilizer bar link nuts",
    "Torque for tie-rod end nut",
    "Torque for upper arm to frame nuts",
    "Torque for wheel bearing and wheel hub bolts",
    "Torque for wheel speed sensor bolt",
    "Torque for brake disc shield bolts",
    "Torque for brake hose bracket bolt",
    "Torque for wheel lug nuts",
]

KEYWORDS = {
    0:  ["lower arm", "350", "nm"],
    1:  ["lower ball joint", "175", "nm"],
    2:  ["upper ball joint", "115"],
    3:  ["shock absorber", "lower", "90"],
    4:  ["shock absorber", "upper", "63"],
    5:  ["shock rod", "55"],
    6:  ["stabilizer bar", "bracket", "55"],
    7:  ["stabilizer bar", "link", "70"],
    8:  ["tie-rod", "115"],
    9:  ["upper arm", "frame", "150"],
    10: ["wheel bearing", "hub", "175"],
    11: ["wheel speed sensor", "18"],
    12: ["brake disc shield", "17"],
    13: ["brake hose bracket", "12"],
    14: ["wheel nut", "150"]
}

def get_relevant_indices(query_idx, chunks):
    kws = KEYWORDS[query_idx]
    return [
        i for i, c in enumerate(chunks)
        if all(k in c.lower() for k in kws)
    ]


# Evaluation Metrics

In [13]:
def recall_k(retrieved, relevant, k):
    return len(set(retrieved[:k]) & set(relevant)) / len(relevant) if relevant else 0

def mrr(retrieved, relevant):
    for rank, idx in enumerate(retrieved, 1):
        if idx in relevant:
            return 1 / rank
    return 0

def precision_k(retrieved, relevant, k):
    return len(set(retrieved[:k]) & set(relevant)) / k if k else 0

def ndcg(retrieved, relevant, k):
    y_true = [1 if i in relevant else 0 for i in retrieved[:k]]
    y_ideal = sorted(y_true, reverse=True)
    return ndcg_score([y_ideal], [y_true], k=k) if any(y_true) else 0


# Retriever Evaluation Function

In [14]:
def evaluate_retriever(retriever, chunks, k=5):
    scores = {
        "recall": [],
        "mrr": [],
        "precision": [],
        "ndcg": [],
        "latency": []
    }

    for qi, query in enumerate(queries):
        start = time()
        docs = retriever.invoke(query)
        latency = time() - start

        retrieved_texts = [d.page_content for d in docs]
        retrieved_indices = [
            i for i, c in enumerate(chunks)
            if c in retrieved_texts
        ]

        relevant = get_relevant_indices(qi, chunks)

        scores["recall"].append(recall_k(retrieved_indices, relevant, k))
        scores["mrr"].append(mrr(retrieved_indices, relevant))
        scores["precision"].append(precision_k(retrieved_indices, relevant, k))
        scores["ndcg"].append(ndcg(retrieved_indices, relevant, k))
        scores["latency"].append(latency)

    return {m: np.mean(v) for m, v in scores.items()}


# Chunking × Retriever Comparison

To study the combined effect of chunking and retrieval, each chunking strategy is paired with every retriever.
With 3 chunking methods and 4 retrievers, this results in 12 total configurations.

## 🔹 Fixed Chunking
- **Fixed + BM25**
- **Fixed + Dense**
- **Fixed + Hybrid**
- **Fixed + Reranked Hybrid**

## 🔹 Recursive Chunking
- **Recursive + BM25**
- **Recursive + Dense**
- **Recursive + Hybrid**
- **Recursive + Reranked Hybrid**

## 🔹 Semantic Chunking
- **Semantic + BM25**
- **Semantic + Dense**
- **Semantic + Hybrid**
- **Semantic + Reranked Hybrid**

---

**Evaluation Metrics:**  
Each configuration is evaluated using:
- Recall@k
- MRR (Mean Reciprocal Rank)
- Precision@k
- NDCG@k (Normalized Discounted Cumulative Gain)
- Latency

These metrics ensure a **fair and comprehensive comparison** across all chunking and retrieval strategies.

In [15]:
results = {}

for cname, chunk_fn in chunking_methods.items():
    chunks = chunk_fn(full_text)
    print(f"{cname}: {len(chunks)} chunks")

    bm25 = create_bm25(chunks)
    dense = create_dense(chunks)
    hybrid = create_hybrid(bm25, dense)
    reranked = create_reranked(hybrid)

    retrievers = {
        "bm25": bm25,
        "dense": dense,
        "hybrid": hybrid,
        "reranked": reranked
    }

    for rname, retriever in retrievers.items():
        metrics = evaluate_retriever(retriever, chunks)
        results[(cname, rname)] = metrics


fixed: 2045 chunks
recursive: 2130 chunks
semantic: 1880 chunks


# Final Comparison Results

In [16]:
df = pd.DataFrame(results).T
df.index = pd.MultiIndex.from_tuples(
    df.index,
    names=["chunking", "retriever"]
)
df

recall       mrr  precision      ndcg   latency
chunking  retriever                                                   
fixed     bm25       0.476984  0.548889   0.333333  0.656420  0.004998
          dense      0.321398  0.543333   0.253333  0.638409  0.017513
          hybrid     0.434890  0.541667   0.306667  0.599131  0.022702
          reranked   0.505525  0.613333   0.360000  0.716488  0.697157
recursive bm25       0.499206  0.548889   0.346667  0.641039  0.005392
          dense      0.308700  0.548889   0.253333  0.653790  0.017819
          hybrid     0.346001  0.539444   0.280000  0.563227  0.023041
          reranked   0.537271  0.613333   0.386667  0.701107  0.691394
semantic  bm25       0.302222  0.266667   0.146667  0.353850  0.004282
          dense      0.100000  0.116667   0.053333  0.135045  0.017713
          hybrid     0.313333  0.285556   0.146667  0.353850  0.023258
          reranked   0.335556  0.327778   0.173333  0.420516  0.909267

We adopt **Recursive Chunking + Reranked Hybrid Retrieval** as our final configuration, based on evaluation results showing it delivers the best **Recall@k, MRR, and Precision@k**, while achieving second-best **NDCG@k**. Latency is not a primary concern for this application.